In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many



---

## **MODEL: cross-encoder/nli-deberta-v3-large**

https://huggingface.co/cross-encoder/nli-deberta-v3-large

In [ ]:
import pandas as pd

In [ ]:
import spacy
!python -m spacy download es_core_news_sm

# Cargar el modelo de spaCy para tokenizar oraciones
nlp = spacy.load("es_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 25.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Función para obtener los enunciados con las keywords
def extract_keyword_sentences(row):
    keywords = set(row['keyword'].split(', '))  # Convertir las keywords en un conjunto para eliminar duplicados
    text = row['text']
    doc = nlp(text)
    found_sentences = set()  # Conjunto para almacenar las oraciones encontradas
    for sentence in doc.sents:
        # Verificar si la oración contiene al menos una de las keywords
        if any(keyword.lower() in sentence.text.lower() for keyword in keywords):
            found_sentences.add(sentence.text.strip())  # Agregar la oración al conjunto de oraciones encontradas
    return '. '.join(found_sentences) if found_sentences else None  # Unir las oraciones encontradas en un solo texto

In [ ]:
# Cargar los datos de entrenamiento
data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', nrows=1800, usecols=[0, 1, 2, 3], engine='python')
# Filtrar y actualizar el texto con los enunciados que contienen las keywords
data['text'] = data.apply(lambda row: extract_keyword_sentences(row), axis=1)
datatext = data['text'].astype(str)

print(len(datatext))

1800


In [ ]:
p1 = datatext[0:100]
print(len(p1))

100


In [ ]:
p2 = datatext[0:10]
print(len(p2))
# Convertir cada texto en el formato deseado
formatted_texts_2 = p2.apply(lambda text: f"('{text}')")
print(formatted_texts_2)

10
0    ('Afterwards, I want to make a run at young lo...
1    ('Since I was still semi-anxious, I didnt real...
2    ('Anyway! I've been partying a few times but a...
3    ('We met because she and I were the last two t...
4    ('So anyway I stepped outside to speak to them...
5    ('Imagine X guy running up to you suddenly: "H...
6    ('That made everything worse bc she was my onl...
7    ('Sitting at the house while my crush is on a ...
8    ('When I imagine myself as an older lady, I pi...
9    ('A yes or no question and yet here i am a wee...
Name: text, dtype: object


In [ ]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/nli-deberta-v3-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Realizar las predicciones para cada texto individualmente
predictions_0 = []
for text in formatted_texts_2:
    score = model.predict([text])
    max_index = score.argmax()
    label_mapping = ['related to an outdoor space', 'not related to an outdoor space']

    if max_index < len(label_mapping):
        label = label_mapping[max_index]

    predictions_0.append(label)

# Imprimir las predicciones
print(predictions_0)


['not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space']


In [ ]:
# Contar el número de predicciones para cada clase
num_outdoor_space = predictions_0.count('related to an outdoor space')
num_not_outdoor_space = predictions_0.count('not related to an outdoor space')

# Imprimir el número de predicciones para cada clase
print("Número de predicciones 'related to an outdoor space':", num_outdoor_space)
print("Número de predicciones 'not related to an outdoor space':", num_not_outdoor_space)

Número de predicciones 'related to an outdoor space': 0
Número de predicciones 'not related to an outdoor space': 10




---

## **MODEL: MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli**

https://huggingface.co/MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli?candidate_labels=positive%2C+negative%2C+neutral%2C+unrelated&multi_class=false&text=Flirting+%3F%3F.+hi+people+of+SA+%21%21%21+I%27m+pretty+new+here%2C+and+I+need+some+advice...+Okay%2C+I+am+a+17F+and+I+like+to+go+on+walks.+I+started+like+a+month+ago%2C+and+I+really+enjoy+having+that+time+to+myself+to+just+think+and+breath+%28I+live+by+the+ocean%2C+so+beach+walks%21%29+Anyways%2C+pretty+much+every+day+I+go+on+a+walk+I+get+a+guy+flirting+with+me+or+whistling%2C+and+I+don%27t+really+know+how+to+deal+with+it.+Like%2C+I%27m+not+wearing+anything+provocative%2C+I+just+want+to+walk+in+the+sand+and+look+at+cute+dogs.+Idk+what+to+do%2C+please+does+someone+have+any+advice%3F+Sidenote+%3A+if+it%27s+a+guy+I+might+be+into+and+he%27s+flirting%2C+what+should+I+do%3F+I%27m+so+awkward+I+never+know+what+to+say+Thanks+%21%21%21

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli")


config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/870M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/395 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions = []
for text in formatted_texts_2:
    output = classifier(text, candidate_labels=['related to an outdoor space', 'not related to an outdoor space'], multi_label=False)
    predictions.append(output)
    print(output)

print(predictions)

{'sequence': "('Afterwards, I want to make a run at young love, but since I've never had any experience, I feel like I'll be building a bridge that will take a few years to build, and by that point I would have missed out.')", 'labels': ['not related to an outdoor space', 'related to an outdoor space'], 'scores': [0.9981882572174072, 0.0018117824802175164]}
{'sequence': "('Since I was still semi-anxious, I didnt really talk anything besides saying hi, sup, and telling her that its boring being home, that I can be shy, etc, she suggested that I could hang around outside but I didnt know if she could hang with')", 'labels': ['related to an outdoor space', 'not related to an outdoor space'], 'scores': [0.9992541074752808, 0.0007459039334207773]}
{'sequence': "('Anyway! I've been partying a few times but as it turns out, it's an incredibly expensive hobby in the long run and I'm not exactly swimming in money.. Almost everything costs money (who woulda thought..), be it going out, watching 

In [ ]:
# Contar el total de clasificaciones como 'outdoor space' y 'not outdoor space'
total_outdoor_space = sum(1 for pred in predictions if pred['labels'][0] == 'related to an outdoor space')
total_not_outdoor_space = sum(1 for pred in predictions if pred['labels'][0] == 'not related to an outdoor space')

# Imprimir los resultados
print("Total de clasificados como 'related to an outdoor space':", total_outdoor_space)
print("Total de clasificados como 'not related to an outdoor space':", total_not_outdoor_space)

Total de clasificados como 'related to an outdoor space': 7
Total de clasificados como 'not related to an outdoor space': 3




---

## **MODEL: alexandrainst/scandi-nli-large**

https://huggingface.co/alexandrainst/scandi-nli-large?candidate_labels=positive%2C+negative%2C+neutral%2C+unrelated+outdoor+space&multi_class=false&text=it%27s+july+and+i+want+to+apply+for+a+summer+job.+i+only+have+till+august+27th+and+i+do+have+an+interview+tomorrow%2C+but+theres+a+chance+i+wont+get+the+job.+i+want+to+apply+to+jobs+but+im+scared+to+piss+off+people+or+be+humiliated+cuz+its+so+late..+well+im+gonna+bike+out+to+stores+and+see+what+happens.

In [ ]:
from transformers import pipeline
classifier = pipeline(
    "zero-shot-classification",
    model="alexandrainst/scandi-nli-large",
)


config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/395k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_3 = []
for text in formatted_texts_2:
    output_3 = classifier(text, candidate_labels=['related to an outdoor space', 'not related to an outdoor space'])
    predictions_3.append(output_3)
    print(output_3)

print(predictions_3)



{'sequence': "('Afterwards, I want to make a run at young love, but since I've never had any experience, I feel like I'll be building a bridge that will take a few years to build, and by that point I would have missed out.')", 'labels': ['related to an outdoor space', 'not related to an outdoor space'], 'scores': [0.7587841153144836, 0.24121585488319397]}
{'sequence': "('Since I was still semi-anxious, I didnt really talk anything besides saying hi, sup, and telling her that its boring being home, that I can be shy, etc, she suggested that I could hang around outside but I didnt know if she could hang with')", 'labels': ['related to an outdoor space', 'not related to an outdoor space'], 'scores': [0.9253193140029907, 0.07468070834875107]}
{'sequence': "('Anyway! I've been partying a few times but as it turns out, it's an incredibly expensive hobby in the long run and I'm not exactly swimming in money.. Almost everything costs money (who woulda thought..), be it going out, watching a mo

In [ ]:
# Contar el total de clasificaciones como 'outdoor space' y 'not outdoor space'
total_outdoor_space = sum(1 for pred in predictions_3 if pred['labels'][0] == 'related to an outdoor space')
total_not_outdoor_space = sum(1 for pred in predictions_3 if pred['labels'][0] == 'not related to an outdoor space')

# Imprimir los resultados
print("Total de clasificados como 'related to an outdoor space':", total_outdoor_space)
print("Total de clasificados como 'not related to an outdoor space':", total_not_outdoor_space)

Total de clasificados como 'related to an outdoor space': 9
Total de clasificados como 'not related to an outdoor space': 1




---

## **MODEL: sileod/deberta-v3-base-tasksource-nli**

https://huggingface.co/sileod/deberta-v3-base-tasksource-nli?candidate_labels=positive%2C+negative%2C+neutral%2C+unrelated&multi_class=false&text=Flirting+%3F%3F.+hi+people+of+SA+%21%21%21+I%27m+pretty+new+here%2C+and+I+need+some+advice...+Okay%2C+I+am+a+17F+and+I+like+to+go+on+walks.+I+started+like+a+month+ago%2C+and+I+really+enjoy+having+that+time+to+myself+to+just+think+and+breath+%28I+live+by+the+ocean%2C+so+beach+walks%21%29+Anyways%2C+pretty+much+every+day+I+go+on+a+walk+I+get+a+guy+flirting+with+me+or+whistling%2C+and+I+don%27t+really+know+how+to+deal+with+it.+Like%2C+I%27m+not+wearing+anything+provocative%2C+I+just+want+to+walk+in+the+sand+and+look+at+cute+dogs.+Idk+what+to+do%2C+please+does+someone+have+any+advice%3F+Sidenote+%3A+if+it%27s+a+guy+I+might+be+into+and+he%27s+flirting%2C+what+should+I+do%3F+I%27m+so+awkward+I+never+know+what+to+say+Thanks+%21%21%21

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",model="sileod/deberta-v3-base-tasksource-nli")


config.json:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_5 = []
for text in formatted_texts_2:
    output_5 = classifier(text, candidate_labels=['related to an outdoor space', 'not related to an outdoor space'])
    predictions_5.append(output_5)
    print(output_5)



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': "('Afterwards, I want to make a run at young love, but since I've never had any experience, I feel like I'll be building a bridge that will take a few years to build, and by that point I would have missed out.')", 'labels': ['not related to an outdoor space', 'related to an outdoor space'], 'scores': [0.7459529042243958, 0.2540470361709595]}
{'sequence': "('Since I was still semi-anxious, I didnt really talk anything besides saying hi, sup, and telling her that its boring being home, that I can be shy, etc, she suggested that I could hang around outside but I didnt know if she could hang with')", 'labels': ['related to an outdoor space', 'not related to an outdoor space'], 'scores': [0.8474230170249939, 0.15257695317268372]}
{'sequence': "('Anyway! I've been partying a few times but as it turns out, it's an incredibly expensive hobby in the long run and I'm not exactly swimming in money.. Almost everything costs money (who woulda thought..), be it going out, watching a mov

In [ ]:
# Contar el total de clasificaciones como 'outdoor space' y 'not outdoor space'
total_outdoor_space = sum(1 for pred in predictions_5 if pred['labels'][0] == 'related to an outdoor space')
total_not_outdoor_space = sum(1 for pred in predictions_5 if pred['labels'][0] == 'not related to an outdoor space')

# Imprimir los resultados
print("Total de clasificados como 'related to an outdoor space':", total_outdoor_space)
print("Total de clasificados como 'not related to an outdoor space':", total_not_outdoor_space)

Total de clasificados como 'related to an outdoor space': 6
Total de clasificados como 'related to an outdoor space': 4




---

## **MODEL: facebook/bart-large-mnli**

https://huggingface.co/facebook/bart-large-mnli?candidate_labels=positive%2C+negative%2C+neutral%2C+unrelated&multi_class=false&text=it%27s+july+and+i+want+to+apply+for+a+summer+job.+i+only+have+till+august+27th+and+i+do+have+an+interview+tomorrow%2C+but+theres+a+chance+i+wont+get+the+job.+i+want+to+apply+to+jobs+but+im+scared+to+piss+off+people+or+be+humiliated+cuz+its+so+late..+well+im+gonna+bike+out+to+stores+and+see+what+happens.

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_6 = []
for text in formatted_texts_2:
    output_6 = classifier(text, candidate_labels=['related to an outdoor space', 'not related to an outdoor space'])
    predictions_6.append(output_6)
    print(output_6)

{'sequence': "('Afterwards, I want to make a run at young love, but since I've never had any experience, I feel like I'll be building a bridge that will take a few years to build, and by that point I would have missed out.')", 'labels': ['not related to an outdoor space', 'related to an outdoor space'], 'scores': [0.5164795517921448, 0.4835204780101776]}
{'sequence': "('Since I was still semi-anxious, I didnt really talk anything besides saying hi, sup, and telling her that its boring being home, that I can be shy, etc, she suggested that I could hang around outside but I didnt know if she could hang with')", 'labels': ['related to an outdoor space', 'not related to an outdoor space'], 'scores': [0.9956598281860352, 0.004340145271271467]}
{'sequence': "('Anyway! I've been partying a few times but as it turns out, it's an incredibly expensive hobby in the long run and I'm not exactly swimming in money.. Almost everything costs money (who woulda thought..), be it going out, watching a mo

In [ ]:
# Contar el total de clasificaciones como 'outdoor space' y 'not outdoor space'
total_outdoor_space = sum(1 for pred in predictions_6 if pred['labels'][0] == 'related to an outdoor space')
total_not_outdoor_space = sum(1 for pred in predictions_6 if pred['labels'][0] == 'not related to an outdoor space')

# Imprimir los resultados
print("Total de clasificados como 'related to an outdoor space':", total_outdoor_space)
print("Total de clasificados como 'not related to an outdoor space':", total_not_outdoor_space)

Total de clasificados como 'related to an outdoor space': 7
Total de clasificados como 'not related to an outdoor space': 3




---

## **MODEL: MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli**

https://huggingface.co/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli?candidate_labels=positive%2C+negative%2C+neutral%2C+unrelated&multi_class=false&text=Flirting+%3F%3F.+hi+people+of+SA+%21%21%21+I%27m+pretty+new+here%2C+and+I+need+some+advice...+Okay%2C+I+am+a+17F+and+I+like+to+go+on+walks.+I+started+like+a+month+ago%2C+and+I+really+enjoy+having+that+time+to+myself+to+just+think+and+breath+%28I+live+by+the+ocean%2C+so+beach+walks%21%29+Anyways%2C+pretty+much+every+day+I+go+on+a+walk+I+get+a+guy+flirting+with+me+or+whistling%2C+and+I+don%27t+really+know+how+to+deal+with+it.+Like%2C+I%27m+not+wearing+anything+provocative%2C+I+just+want+to+walk+in+the+sand+and+look+at+cute+dogs.+Idk+what+to+do%2C+please+does+someone+have+any+advice%3F+Sidenote+%3A+if+it%27s+a+guy+I+might+be+into+and+he%27s+flirting%2C+what+should+I+do%3F+I%27m+so+awkward+I+never+know+what+to+say+Thanks+%21%21%21

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_7 = []
for text in formatted_texts_2:
    output_7 = classifier(text, candidate_labels=['related to an outdoor space', 'not related to an outdoor space'])
    predictions_7.append(output_7)
    print(output_7)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': "('Afterwards, I want to make a run at young love, but since I've never had any experience, I feel like I'll be building a bridge that will take a few years to build, and by that point I would have missed out.')", 'labels': ['not related to an outdoor space', 'related to an outdoor space'], 'scores': [0.9956103563308716, 0.004389618523418903]}
{'sequence': "('Since I was still semi-anxious, I didnt really talk anything besides saying hi, sup, and telling her that its boring being home, that I can be shy, etc, she suggested that I could hang around outside but I didnt know if she could hang with')", 'labels': ['related to an outdoor space', 'not related to an outdoor space'], 'scores': [0.9797137975692749, 0.020286206156015396]}
{'sequence': "('Anyway! I've been partying a few times but as it turns out, it's an incredibly expensive hobby in the long run and I'm not exactly swimming in money.. Almost everything costs money (who woulda thought..), be it going out, watching a 

In [ ]:
# Contar el total de clasificaciones como 'outdoor space' y 'not outdoor space'
total_outdoor_space = sum(1 for pred in predictions_7 if pred['labels'][0] == 'related to an outdoor space')
total_not_outdoor_space = sum(1 for pred in predictions_7 if pred['labels'][0] == 'not related to an outdoor space')

# Imprimir los resultados
print("Total de clasificados como 'related to an outdoor space':", total_outdoor_space)
print("Total de clasificados como 'not related to an outdoor space':", total_not_outdoor_space)

Total de clasificados como 'related to an outdoor space': 7
Total de clasificados como 'not related to an outdoor space': 3




---

## **MODEL: MoritzLaurer/mDeBERTa-v3-base-mnli-xnli**

https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-mnli-xnli?candidate_labels=positive%2C+negative%2C+neutral%2C+unrelated&multi_class=false&text=it%27s+july+and+i+want+to+apply+for+a+summer+job.+i+only+have+till+august+27th+and+i+do+have+an+interview+tomorrow%2C+but+theres+a+chance+i+wont+get+the+job.+i+want+to+apply+to+jobs+but+im+scared+to+piss+off+people+or+be+humiliated+cuz+its+so+late..+well+im+gonna+bike+out+to+stores+and+see+what+happens.

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")


config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_8 = []
for text in formatted_texts_2:
    output_8 = classifier(text, candidate_labels=['related to an outdoor space', 'not related to an outdoor space'])
    predictions_8.append(output_8)
    print(output_8)

{'sequence': "('Afterwards, I want to make a run at young love, but since I've never had any experience, I feel like I'll be building a bridge that will take a few years to build, and by that point I would have missed out.')", 'labels': ['related to an outdoor space', 'not related to an outdoor space'], 'scores': [0.9599166512489319, 0.040083326399326324]}
{'sequence': "('Since I was still semi-anxious, I didnt really talk anything besides saying hi, sup, and telling her that its boring being home, that I can be shy, etc, she suggested that I could hang around outside but I didnt know if she could hang with')", 'labels': ['related to an outdoor space', 'not related to an outdoor space'], 'scores': [0.9958010315895081, 0.0041989656165242195]}
{'sequence': "('Anyway! I've been partying a few times but as it turns out, it's an incredibly expensive hobby in the long run and I'm not exactly swimming in money.. Almost everything costs money (who woulda thought..), be it going out, watching a

In [ ]:
# Contar el total de clasificaciones como 'outdoor space' y 'not outdoor space'
total_outdoor_space = sum(1 for pred in predictions_8 if pred['labels'][0] == 'related to an outdoor space')
total_not_outdoor_space = sum(1 for pred in predictions_8 if pred['labels'][0] == 'not related to an outdoor space')

# Imprimir los resultados
print("Total de clasificados como 'related to an outdoor space':", total_outdoor_space)
print("Total de clasificados como 'not related to an outdoor space':", total_not_outdoor_space)

Total de clasificados como 'related to an outdoor space': 10
Total de clasificados como 'not related to an outdoor space': 0





---


---



---

## **PRUEBA CON TODOS LOS DATOS DEL MEJOR ANTERIOR: 'cross-encoder/nli-deberta-v3-large'**

In [ ]:
pTOTAL = datatext
print(len(pTOTAL))
# Convertir cada texto en el formato deseado
formatted_texts_TOTAL = pTOTAL.apply(lambda text: f"('{text}')")
print(len(formatted_texts_TOTAL))

1800
1800


In [ ]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/nli-deberta-v3-large')

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Realizar las predicciones para cada texto individualmente
predictions_TOTAL = []
for text in formatted_texts_TOTAL:
    score = model.predict([text])
    max_index = score.argmax()
    label_mapping = ['related to an outdoor space', 'not related to an outdoor space']

    if max_index < len(label_mapping):
        label = label_mapping[max_index]

    predictions_TOTAL.append(label)

# Imprimir las predicciones
print(predictions_TOTAL)


['not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'not related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'related to an outdoor space', 'rela

In [ ]:
# Contar el número de predicciones para cada clase
num_outdoor_space = predictions_TOTAL.count('related to an outdoor space')
num_not_outdoor_space = predictions_TOTAL.count('not related to an outdoor space')

# Imprimir el número de predicciones para cada clase
print("Número de predicciones 'related to an outdoor space':", num_outdoor_space)
print("Número de predicciones 'not related to an outdoor space':", num_not_outdoor_space)

from sklearn.metrics import classification_report
data['label'] = data['label'].apply(lambda x: 1 if x != 0 else 0)
# Obtener las etiquetas reales del conjunto de datos
y_true = data['label']

# Convertir las etiquetas predichas a números según el formato especificado en tu pregunta
label_map = {
    'related to an outdoor space': 1,
    'not related to an outdoor space': 0
}

# Mapear las etiquetas predichas al formato numérico
y_pred = [label_map[label] for label in predictions_TOTAL]

# Obtener el classification report
print(classification_report(y_true, y_pred))


Número de predicciones 'related to an outdoor space': 1496
Número de predicciones 'not related to an outdoor space': 304
              precision    recall  f1-score   support

           0       0.65      0.18      0.28      1131
           1       0.38      0.84      0.52       669

    accuracy                           0.42      1800
   macro avg       0.52      0.51      0.40      1800
weighted avg       0.55      0.42      0.37      1800

